In [27]:
from gen_finetune.run_finetune_experiment import get_dataset, prep_train_dataset, prep_val_dataset
from pathlib import Path
import json
import transformers
from transformers import pipeline
import torch
import os

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version used by PyTorch: {torch.version.cuda}")

torch.cuda.init()
print("After init - CUDA available:", torch.cuda.is_available())

%reload_ext autoreload
%autoreload 2


PyTorch version: 2.7.1+cu126
CUDA available: True
CUDA version used by PyTorch: 12.6
After init - CUDA available: True


In [10]:
data_folder = Path("data/title_and_first_sen")
dataset, task_description = get_dataset(str(data_folder / "data-test.jsonl"), str(data_folder / "data-task.json"))
print(dataset)
print(dataset[0]["task_input_a"])
print(dataset[0]["task_answer_a"])

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['task_input_a', 'task_input_b', 'task_answer_a', 'task_answer_b'],
    num_rows: 2601
})
System: You are OLMo 2, a helpful and harmless AI Assistant built by the Allen Institute for AI. ---- User: [Update] My family was robbed and we know who did it. How do I comfort my family and confront the robber? This is an update to my original post, found here:
https://www
Assistant: <reddit>relationships</reddit>


In [30]:
pipeline_llama = pipeline(
    task="text-generation",
    model="huggyllama/llama-7b",
    torch_dtype=torch.float16,
    device=0
)
pipeline_llama("Plants create energy through a process known as")
del pipeline_llama

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [25]:
print(task_description)
print(dataset)
dataset, task_description = get_dataset(str(data_folder / "data-test.jsonl"), str(data_folder / "data-task.json"))
val_data = prep_val_dataset(dataset, task_description)
val_data_cross = prep_val_dataset(dataset, task_description, cross=True)

print(val_data[2]["label"])
print(val_data[3]["label"])
print(val_data_cross)
print(val_data_cross[2]["label"])
print(val_data_cross[3]["label"])

print(dataset[0])

TaskDescription(prompt_a='Which subreddit does this belong to? -----', prompt_b='Continue the story. -----', tag_a='reddit', tag_b='story')
Dataset({
    features: ['task_input_a', 'task_input_b', 'task_answer_a', 'task_answer_b'],
    num_rows: 2601
})


Map:   0%|          | 0/2601 [00:00<?, ? examples/s]

Map:   0%|          | 0/2601 [00:00<?, ? examples/s]

Assistant: <reddit>AmItheAsshole</reddit>
Assistant: <story>  My middle name is more obviously a girls name BUT I love my first name and I don't love my middle name </story>
Dataset({
    features: ['generation', 'label', 'task'],
    num_rows: 5202
})
Assistant: <reddit>AmItheAsshole</reddit>
Assistant: <story>  My middle name is more obviously a girls name BUT I love my first name and I don't love my middle name </story>
{'task_input_a': 'System: You are OLMo 2, a helpful and harmless AI Assistant built by the Allen Institute for AI. ---- User: [Update] My family was robbed and we know who did it. How do I comfort my family and confront the robber? This is an update to my original post, found here:\nhttps://www', 'task_input_b': "System: You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your respon

In [31]:
exp_dir = Path("/workspace/chunky-experiments/experiments/Fox_2025-06-21_19-40-38")
checkpoint = exp_dir / "model"
os.listdir(checkpoint)
checkpoint_path = checkpoint / "checkpoint-1000"

pipeline_test = pipeline(
    task="text-generation",
    model=str(checkpoint_path),
    torch_dtype=torch.float16,
    device=0
)


Device set to use cuda:0


In [43]:
def process_in_batches(data, pipeline, batch_size=8):
    results = []
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        batch_inputs = [item["generation"] for item in batch]
        batch_results = pipeline(batch_inputs)
        results.extend(batch_results)
    return results

# Usage
batch_results = process_in_batches(val_data, pipeline_test, batch_size=8)

# Process results
for idx, result in enumerate(batch_results):
    print(f"Label {idx}: {val_data[idx]['label']}")
    generated_text = result[0]["generated_text"][len(val_data[idx]["generation"])+1:]
    print(f"Generated: {generated_text}")

Assistant: <reddit>AmItheAsshole</reddit>


TypeError: list indices must be integers or slices, not str

In [41]:
def count_tags(generated_text: str):
    tag_a = "<story>"
    tag_b = "<reddit>"
    return {"tag_a": tag_a in generated_text, "tag_b": tag_b in generated_text}


def eval_dataset(dataset: datasets.Dataset, pipeline: pipeline):

    for i in range(len(dataset)):
        label = dataset[i]["label"]
        generation = pipeline(dataset[i]["generation"])[0]["generated_text"][len(dataset[i]["generation"])+1:]
        print(label)
        print(generation)
        print(eval_task_a(label, generation))
        print(eval_task_b(label, generation))